In [48]:
import twitter
import json
import pymongo

#--------------------------------------------------------------------------------------
# The purpose of oauth_login() function is
#    1. Login to the Twitter API
#    2. Return twitter handle necessary for its use in later functions

def oauth_login():
    # Had to go to http://twitter.com/apps/new to create an app and get values
    # for these credentials
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'zmbyPowgnW1QNs9LXjGtNqMcm'
    CONSUMER_SECRET ='wnbgjQQJxidVvnw3CORSw5MVjbOE8KstdXlkPLrH01R9JLAlPn'
    OAUTH_TOKEN = '1529017262-SoyYbWBJWW8Mn0MLuyQLpmViknAiGQg1zVJv3N7'
    OAUTH_TOKEN_SECRET = '4nqZePogkTfNVBJqqb1VN0dpprz8ykeKBDrdxXsxDFYnC'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api
#--------------------------------------------------------------------------------------
# The purpose of save_to_mongo() function is
#    1. Connect to the MongoDB server running on localhost
#    2. Get a reference to database specified in input paramter
#    3. Reference collection specified in input parameter
#    4. Save data to database and return IDs

def save_to_mongo(data, mongo_db, mongo_db_coll, **mongo_conn_kw):
    
    # Connects to the MongoDB server running on 
    # localhost:27017 by default
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    
    # Get a reference to a particular database
    
    db = client[mongo_db]
    
    # Reference a particular collection in the database
    
    coll = db[mongo_db_coll]
    
    # Perform a bulk insert and  return the IDs
    return coll.insert_many(data)
#--------------------------------------------------------------------------------------
# The purpose of load_from_mongo() function is
#    1. Given the db and coll passed in, find data based on criteria if given
#    4. Load data from database and return cursor or item to user

def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False,
                    criteria=None, projection=None, **mongo_conn_kw):
    
    # Optionally, use criteria and projection to limit the data that is 
    # returned as documented in 
    # http://docs.mongodb.org/manual/reference/method/db.collection.find/
    
    # Consider leveraging MongoDB's aggregations framework for more 
    # sophisticated queries.
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    db = client[mongo_db]
    coll = db[mongo_db_coll]
    
    if criteria is None:
        criteria = {}
    
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)

    # Returning a cursor is recommended for large amounts of data
    
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]



#--------------------------------------------------------------------------------------
# The purpose of the get_trends() function is:
#    1. Gather current world trends from twitter
#    2. Gather current US trends from twitter
#    3. Print the trends for viewing (TODO: Do we want to save trending data?)

def get_trends(twitter_api):
    
    # The Yahoo! Where On Earth ID for the entire world is 1.
    # See https://dev.twitter.com/docs/api/1.1/get/trends/place and
    # http://developer.yahoo.com/geo/geoplanet/

    WORLD_WOE_ID = 1
    US_WOE_ID = 23424977

    # Prefix ID with the underscore for query string parameterization.
    # Without the underscore, the twitter package appends the ID value
    # to the URL itself as a special case keyword argument.

    world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
    us_trends = twitter_api.trends.place(_id=US_WOE_ID)


    print ('*** Here are the WORLD TRENDS :')
    print (json.dumps(world_trends, indent=1))
    print ('*** Here are the US TRENDS :')
    print (json.dumps(us_trends, indent=1))

#--------------------------------------------------------------------------------------
# OLD CODE - NO LONGER USED
# The purpose of the getPokemonGoTweets() function is:
#   1. Search twitter for tweets containing the search term '#PokemonGo'
#   2. Save any tweets retrieved in a file named 'pokemonresults.txt'
#
# This was the original function written to get PokemonGo tweets.  It is no longer
# being called by anyone, but was saved to show our first attempt using the polling API
# and writing results to a file rather than a database.

def getPokemonGoTweets(twitter_api):
    
       
    # Import unquote to prevent url encoding errors in next_results
    from urllib.parse import unquote

    # Set q to the topic of interest for CaseStudy1-Problem1, PokemonGo 

    q = '#PokemonGo' 

    count = 100

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets

    search_results = twitter_api.search.tweets(q=q, count=count)

    statuses = search_results['statuses']


    # Iterate through 5 more batches of results by following the cursor

    for _ in range(5):
        print ("Length of statuses", len(statuses))
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])

        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
    # Dump data to json file in json format    
    with open('pokemonresults.txt', 'w') as outfile:
        json.dump(statuses, outfile)  
    # Note: by opening file using 'with', no need to close() the file. It happens implicitly.
    

#--------------------------------------------------------------------------------------
# The purpose of the twitter_search() function is:
#   1. Search twitter for tweets containing the search term q, defaulting results=200
#      because oAuth users can "only" make 180 search queries per 15 minute interval
#
#   2. return statuses

def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

        

#--------------------------------------------------------------------------------------
# The purpose of the mainDataGathering() function is:
#     1. Login to twitter and get api handle
#     1. Gather desired twitter data and save for future use

def mainDataGathering():
    
    twitter_api = oauth_login()
    print(twitter_api)
    
    # Search twitter for "#PokemonGo" tweets
    q = "#PokemonGo"

    # Use twitter streaming API
    twitter_stream = twitter.TwitterStream(auth=twitter_api.auth)
    
    # See https://dev.twitter.com/docs/streaming-apis
    #stream = twitter_stream.statuses.filter(track=q)
    
    
    #get_trends(twitter_api)

    #getPokemonGoTweets(twitter_api)
    
    # MongoDB---------------------------------------------------
    # Connects to the MongoDB server running on 
    # localhost:27017 by default
    
    #client = pymongo.MongoClient(**mongo_conn_kw)
    client = pymongo.MongoClient('localhost', 27017)
    
    # Get a reference to a particular database
    mongo_db = 'stream_search_results'
    db = client[mongo_db]
    
    # Reference a particular collection in the database
    mongo_db_coll = 'StreamingTweet'
    coll = db[mongo_db_coll]

    
    
    
    #results = twitter_search(twitter_api, q, max_results=10)
    stream = twitter_stream.statuses.filter(track=q)
    for tweet in stream:
        #print(tweet['text'])  
        # Save results to MongoDB
        coll.insert_one(tweet)
        #save_to_mongo(stream, 'stream_search_results', 'StreamingTweet')

    # --------------------------------------------------------------------------------
    # This is an example of how to get some data from the database
    #
    #print('**** Load the DATABASE ***')
    #cursor = load_from_mongo('stream_search_results', q, return_cursor=True)
    #print('**** HERE is some data from the DATABASE ***')
    #for trend in cursor:
        # Print the text
        # print(trend['text'])
        # Print user and location
        #print(trend['user']['location'])
    

    

#--------------------------------------------------------------------------------------
# Call the mainDataGathering() function to perform actions needed for 
# DS501 CaseStudy 1 Problem 1 : Sampling twitter data with streaming api about a topic

mainDataGathering()

#--------------------------------------------------------------------------------------
# Test that the mainDataGathering() worked by displaying a sample result from the file
# This is not necessary and can be commented out if display is not desired here...

#print ("*** Sample PokemonGo output : ")
# Get one sample search result from file
#with open('pokemonresults.txt','r') as json_data:
#    json_statuses = json.load(json_data)
#    print(json.dumps(json_statuses[0], indent=4))
# Note: by opening file using 'with', no need to close() the file. It happens implicitly.

# q = "#PokemonGo"
#load_from_mongo('stream_search_results', q)


#Problem-2

#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

import json
import pymongo

from prettytable import PrettyTable

mongo_db = 'stream_search_results'
mongo_db_coll = 'StreamingTweet'

# --------------------------------------------------------------------------------
# This is an example of how to get some data from the database
#
print('**** Load the DATABASE ***')
#cursor = load_from_mongo('stream_search_results', 'twitter', return_cursor=True)
# Connects to the MongoDB server running on 
# localhost:27017 by default
    
client = pymongo.MongoClient()
    
# Get a reference to a particular database
    
db = client[mongo_db]
    
# Reference a particular collection in the database
coll = db[mongo_db_coll]

cursor = coll.find({})


print('**** HERE is some data from the DATABASE ***')

#for trend in cursor:
    #print(trend['text'])
    #print(trend['user']['location'])
    

     
# This code is based on Example #10-13 in Chapter 9 of Mining the Social Web
    


curr_tweet_texts = [ curr_tweet['text'] 
                     for curr_tweet in cursor ]

cursor = coll.find({})

screen_names = [ user_mention['screen_name'] 
                 for curr_tweet in cursor
                     for user_mention in curr_tweet['entities']['user_mentions'] ]

cursor = coll.find({})

hashtags = [ hashtag['text'] 
             for curr_tweet in cursor
                 for hashtag in curr_tweet['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in curr_tweet_texts 
              for w in t.split() ]

# Explore the first 5 items for each...

print (json.dumps(curr_tweet_texts[0:5], indent=1))
print (json.dumps(screen_names[0:5], indent=1)) 
print (json.dumps(hashtags[0:5], indent=1))
print (json.dumps(words[0:5], indent=1))

from collections import Counter

print()
print('*** And here are the TOP 30 : ')

for item in [words]:
    pt = PrettyTable(field_names=['Words', 'Count'])
    c = Counter(item)
    #print (c.most_common()[:30]) # top 30
    [ pt.add_row(kv) for kv in c.most_common()[:30] ]
    pt.align['Words'], pt.align['Count'] = 'l', 'r'
    print(pt)


 Got tweets ...


TypeError: string indices must be integers

In [3]:
import pymongo
import json

def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False,
                    criteria=None, projection=None, **mongo_conn_kw):
    
    # Optionally, use criteria and projection to limit the data that is 
    # returned as documented in 
    # http://docs.mongodb.org/manual/reference/method/db.collection.find/
    
    # Consider leveraging MongoDB's aggregations framework for more 
    # sophisticated queries.
    
    client = pymongo.MongoClient(**mongo_conn_kw)
    db = client[mongo_db]
    coll = db[mongo_db_coll]
    
    if criteria is None:
        criteria = {}
    
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)

    # Returning a cursor is recommended for large amounts of data
    
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]

mdb='search_results'
mcoll='#PokemonGo'
cursor = load_from_mongo(mdb, mcoll, return_cursor=True)
    
print('**** HERE is the data in the DATABASE ***')
'''for trend in cursor:
    #print(trend['text'])
    print(trend['user']['location'])
    print("---------------------")
    screen_names = trend['user']['screen_name'] 
                        # for user_mention in trend['user']]'''
    
i=0    
print ("Out of for")    
#print (screen_names)
print("-------------------------Hashtags-----------------------")
hashtags = [hashtag['text']
                   for trend in cursor
                       #print("---XXX---") 
                       for hashtag in trend['entities']['hashtags']]

#print json.dumps(hashtags[0:5], indent=1)
print(hashtags)

print("-------------------------ScreenNames-----------------------")
screen_names = [screen_name_container['screen_name']
                   for trend in cursor
                       #print("---XXX---") 
                       for screen_name_container in trend['entities']['user_mentions']]

#print json.dumps(hashtags[0:5], indent=1)
print(screen_names)

'''for var in hashtags
    print(var['text'])
    i=i+1
    if i>10
        break;
print("End")'''

**** HERE is the data in the DATABASE ***
Out of for
-------------------------Hashtags-----------------------
['PokemonGO', 'Pokemon', 'PokemonGo', 'Oddish', 'CaisterLifeboatStation', 'PokemonGo', 'PokemonGO', 'CoastalCleanupDay', 'sanleandromarina', 'PokemonGO', 'PokemonGo', 'PokeArcadia', 'Poliwag', 'TwentseWelle', 'PokemonGo', 'Pokewatch', 'PokemonGo', 'SmallBiz', 'GrowthHacking', 'Jynx', 'McDonalds', 'PokemonGo', 'LakeAnna', 'PokemonGo', 'PokeArcadia', 'PokemonGo', 'PokémonGo', 'PokemonGO', 'PokémonGo', 'Machop', 'FergusonPk', 'PokemonGo', 'Tauranga', 'PokemonGo', 'pokémongo', 'gym', 'PokemonGO', 'PokemonGO', 'agario', 'FUNNY', 'TURKEY', 'europe', 'game', 'illuminati', 'POKEMONGO', 'NEW', 'TROLL', 'Diglett', 'PokemonGo', 'pokemongo', 'pokemon', 'PokemonGO', 'pokemongo', 'ポケモンGO', 'PokemonGO', 'PokemonGO', 'PokemonGOPlus', 'Pug', 'Dogs', 'Animals', 'PokemonGo', 'Games', 'PokemonGO', 'Pokemon', 'PokemonGO', 'PokemonGO', 'ポケモンGO', 'PokemonGO', 'フシギダネ', 'ヒトカゲ', 'ゼニガメ', 'ピカチュウ', 'Pokemo

'for var in hashtags\n    print(var[\'text\'])\n    i=i+1\n    if i>10\n        break;\nprint("End")'

In [29]:
from prettytable import PrettyTable

# Get some frequency data
q="#PokemonGo"
twitter_api = oauth_login()
search_results = twitter_search(twitter_api, q, max_results=100)
common_entities = get_common_tweet_entities(search_results)

# Use PrettyTable to create a nice tabular display

pt = PrettyTable(field_names=['Entity', 'Count']) 
[ pt.add_row(kv) for kv in common_entities ]
pt.align['Entity'], pt.align['Count'] = 'l', 'r' # Set column alignment
print (pt)

NameError: name 'get_common_tweet_entities' is not defined